In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.appName("Prata3") \
.getOrCreate()

In [3]:
from pyspark.sql.functions import col, sum, count, when, countDistinct

In [4]:
df = spark.read.csv("dados\df2.csv", sep=";", header=True)

In [5]:
df.show()

+--------------------+--------------------+
|                 URL|         Comentários|
+--------------------+--------------------+
|https://www.merca...|Está muito caro e...|
|https://www.merca...|                NULL|
|https://www.merca...|Eu adoro este tap...|
|https://www.merca...|Absorve bem. Indico.|
|https://www.merca...|Já usei vários ta...|
|https://www.merca...|Excelente qualida...|
|https://www.merca...|O produto é excel...|
|https://www.merca...|Estou sempre comp...|
|https://www.merca...|Muito boa qualida...|
|https://www.merca...|O super secão é o...|
|https://www.merca...|      Já foi melhor.|
|https://www.merca...|              Ótimo.|
|https://www.merca...|Meu cachorro so a...|
|https://www.merca...|Muito bom, não va...|
|https://www.merca...|                Bom.|
|https://www.merca...|  Sempre uso, é bom.|
|https://www.merca...|Muito bom só uso ...|
|https://www.merca...|É um bom produto,...|
|https://www.merca...|        Bom produto.|
|https://www.merca...|Já experim

In [6]:
df.count()

44907

In [7]:
df = df.withColumnRenamed("Comentários", "Comentarios")
df.show()

+--------------------+--------------------+
|                 URL|         Comentarios|
+--------------------+--------------------+
|https://www.merca...|Está muito caro e...|
|https://www.merca...|                NULL|
|https://www.merca...|Eu adoro este tap...|
|https://www.merca...|Absorve bem. Indico.|
|https://www.merca...|Já usei vários ta...|
|https://www.merca...|Excelente qualida...|
|https://www.merca...|O produto é excel...|
|https://www.merca...|Estou sempre comp...|
|https://www.merca...|Muito boa qualida...|
|https://www.merca...|O super secão é o...|
|https://www.merca...|      Já foi melhor.|
|https://www.merca...|              Ótimo.|
|https://www.merca...|Meu cachorro so a...|
|https://www.merca...|Muito bom, não va...|
|https://www.merca...|                Bom.|
|https://www.merca...|  Sempre uso, é bom.|
|https://www.merca...|Muito bom só uso ...|
|https://www.merca...|É um bom produto,...|
|https://www.merca...|        Bom produto.|
|https://www.merca...|Já experim

In [8]:
df.filter(col("Comentarios").isNull()).count()

4042

In [9]:
df.select(count(when(df["Comentarios"].isNull(), True)).alias("Quant_Nulos")).show()

+-----------+
|Quant_Nulos|
+-----------+
|       4042|
+-----------+



In [10]:
df_sem_nulos = df.dropna()

In [11]:
df_sem_nulos.count()

40865

In [12]:
df_sem_nulos.select(count(when(col("Comentarios").isNull(),True)).alias("Quant_Nulos")).show()

+-----------+
|Quant_Nulos|
+-----------+
|          0|
+-----------+



In [13]:
df.select(countDistinct(col("URL"))).show()

+-------------------+
|count(DISTINCT URL)|
+-------------------+
|               3520|
+-------------------+



In [14]:
df_sem_nulos.select(countDistinct(col("URL"))).show()

+-------------------+
|count(DISTINCT URL)|
+-------------------+
|               1937|
+-------------------+



In [16]:
id_urls = spark.read.csv("dados\IDs_URLs.csv", sep = ";", header = True)
df2_tratado = id_urls.join(df_sem_nulos, on="URL", how="inner")
df2_tratado = df2_tratado.drop("URL")
df2_tratado.show()

+----+--------------------+
|  ID|         Comentarios|
+----+--------------------+
|1419|Está muito caro e...|
|1419|Eu adoro este tap...|
|1419|Absorve bem. Indico.|
|1419|Já usei vários ta...|
|1419|Excelente qualida...|
|1419|O produto é excel...|
|1419|Estou sempre comp...|
|1419|Muito boa qualida...|
|1419|O super secão é o...|
|1419|      Já foi melhor.|
|1419|              Ótimo.|
|1419|Meu cachorro so a...|
|1419|Muito bom, não va...|
|1419|                Bom.|
|1419|  Sempre uso, é bom.|
|1419|Muito bom só uso ...|
|1419|É um bom produto,...|
|1419|        Bom produto.|
|1419|Já experimentei d...|
|1419|              Ótimo.|
+----+--------------------+
only showing top 20 rows



In [19]:
df2_tratado.write.parquet("dados\df2_tratado.parquet")

In [17]:
df2_tratado.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Comentarios: string (nullable = true)

